In [1]:
!pip install wandb boto3 pyarrow

In [2]:
# Import required libraries

from utils import configure_notebook_session
import random
import pandas as pd
import wandb

In [3]:
# Configure some constants for the demo

PROJECT_NAME = 'diabetes-vaccines-notebook'
NOTEBOOK_NAME = 'train-data.ipynb'
ARTIFACT_NAME = 'training-data'
RESOURCES = ['Patient', 'Immunization', 'Condition']
START_BIRTH_DATE = '1920-01-01'
END_BIRTH_DATE = '2020-01-01'
IMMUNIZATION_CODE = '08' # Hepatitis B
CONDITION_CODE = '73211009' # Diabetes

# Configure notebook and login to wandb
configure_notebook_session(NOTEBOOK_NAME)
wandb.login()

wandb: You can find your API key in your browser here: http://wandb:8080/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR Host must be a url in the form https://some.address.com, received http://wandb:8080


True

In [4]:
# Start a new wandb run to track this script
run = wandb.init(
  project=PROJECT_NAME,
  config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
  }
)

wandb: Currently logged in as: louismullie. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [5]:
artifact = run.use_artifact('training-data:latest', type='dataset')
artifact_dir = artifact.download()
df = pd.read_parquet(artifact_dir, engine='pyarrow')

print(df.head())

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


CommError: Project louismullie/diabetes-vaccines-notebook does not contain artifact: "training-data:latest"

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
# Simulate training
epochs = 100
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    run.log({"acc": acc, "loss": loss})

In [ ]:
# Save model as a Wandb artifact
file = './model.pt' # model.save("path/to/model.pt")
open(file, 'a').close()

art = wandb.Artifact(f'{PROJECT_NAME}-{wandb.run.id}', type="model")
art.add_file("./model.pt", "model.pt")
run.log_artifact(art)

In [ ]:
# Finish tracking
run.finish()